## Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
#%matplotlib notebook # required for training plots

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import os                                                                       
import numpy as np                                                              
import tensorflow as tf                                                         
import data.data_picker as dp                                                   
import utils.plot_functions as pf                                               
import utils.image_processing as ip                                             
import utils.notebook as nb

import matplotlib.gridspec as gsp

## Model parameters

In [ ]:
params = {
  ## Model params
  "out_dir": os.path.expanduser("~")+"/Work/Projects/ICAstrongPCA/outputs/",
  "chk_dir": os.path.expanduser("~")+"/Work/Projects/ICAstrongPCA/checkpoints/",
  "data_dir": os.path.expanduser("~")+"/Work/Datasets/",
  "load_chk": True,
  "load_cov": False,
  "update_interval": 500,
  "device": "/gpu:0",
  "prior": "laplacian",
  "learning_rate": 0.01,
  "num_neurons": int(16**2),
  "eta": 0.001/0.03, #dt/tau
  "eps": 1e-12,
  "max_cp_to_keep": 2,
  ## Data params
  "data_type": "vanhateren",
  "rand_state": np.random.RandomState(12345),
  "num_images": 100,
  "num_batches": int(1e5), #Total dataset size is num_batches*batch_size
  "batch_size": 100,
  "patch_edge_size": 16,
  "overlapping_patches": True,
  "patch_variance_threshold": 1e-6,
  "conv": False,
  "whiten_images": False,
  "contrast_normalize": False,
  ## Fourier analysis params
  "ft_padding": 20,
  ## Pooling params
  "cov_num_images":int(1e7),
  "num_pooling_dims": 24, #K
  ## Visualization params
  "num_pooling_filters":50,
  "num_connected_weights":256}

## Calculated params
params["epoch_size"] = params["batch_size"] * params["num_batches"]
params["num_training_steps"] = 20000 # params["num_batches"] # Could be set to less, model trains in 20k
params["num_pixels"] = int(params["patch_edge_size"]**2)
params["dataset_shape"] = [int(val)
    for val in [params["epoch_size"], params["num_pixels"]]],
params["phi_shape"] = [params["num_neurons"], params["num_pixels"]]

## Functions for analysis

### Pooling filters function

In [ ]:
def compute_pooling_filters(activity_covariance, num_pooling_dims):
  evals, evecs = np.linalg.eig(activity_covariance)
  sort_indices = np.argsort(evals)[::-1]
  top_vecs = evecs[:, sort_indices[:num_pooling_dims]]
  pooling_filters = np.dot(top_vecs, top_vecs.T)
  return pooling_filters

## Load dataset

In [ ]:
if not params["load_cov"] or not params["load_chk"]: 
  data = dp.get_data(params["data_type"], params)                                 
  params["input_shape"] = [                                                       
      data["train"].num_rows*data["train"].num_cols*data["train"].num_channels]     

## Define graph

In [ ]:
graph = tf.Graph()
with tf.device(params["device"]):
  with graph.as_default():
    with tf.name_scope("placeholders") as scope:
      x = tf.placeholder(tf.float32, shape=[None, params["num_pixels"]],
        name="input_data")

    with tf.name_scope("step_counter") as scope:
      global_step = tf.Variable(0, trainable=False, name="global_step")

    with tf.variable_scope("weights") as scope:
      ## Q matrix from QR decomp is guaranteed to be orthonormal and
      ## non-singular, which prevents a gradient explosion from inverting
      ## the weight matrix.
      Q, R = np.linalg.qr(np.random.standard_normal(params["phi_shape"]))
      phi = tf.get_variable(name="phi", dtype=tf.float32,
        initializer=Q.astype(np.float32), trainable=True)
      phi_inverse = tf.matrix_inverse(phi, name="phi_inverse")

    with tf.name_scope("inference") as scope:
      u = tf.matmul(x, phi_inverse, name="coefficients")
      if params["prior"] == "laplacian":
        a = tf.sign(u)
      else: #It must be laplacian or cauchy
        a = (2*u) / (1 + tf.pow(u, 2.0))
      u_covariance = tf.divide(tf.matmul(tf.transpose(tf.nn.relu(u)), tf.nn.relu(u)),
        tf.to_float(tf.shape(x)[0]), name="u_cov_matrix")

    with tf.name_scope("optimizers") as scope:
      learning_rates = tf.train.exponential_decay(
        learning_rate=params["learning_rate"],
        global_step=global_step,
        decay_steps=int(np.floor(params["num_batches"]*0.8)),
        decay_rate=0.7,
        staircase=True,
        name="phi_annealing_schedule")
      optimizer = tf.train.GradientDescentOptimizer(learning_rates, name="phi_optimizer")
      a_u_avg = tf.divide(tf.matmul(tf.transpose(u), a),                   
        tf.to_float(tf.shape(x)[0]), name="avg_samples")                        
      gradient = -tf.subtract(tf.matmul(a_u_avg, phi), phi, name="phi_gradient")                                                
      update_weights = optimizer.apply_gradients([(gradient, phi)], global_step=global_step)

    full_saver = tf.train.Saver(var_list=[phi], max_to_keep=2)

    with tf.name_scope("summaries") as scope:
      #tf.summary.image("input", tf.reshape(x, [params["batch_size"],
      #  params["patch_edge_size"], params["patch_edge_size"], 1]))
      tf.summary.histogram("u", u)
      tf.summary.histogram("a", a)
      tf.summary.histogram("phi", phi)

    merged_summaries = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(params["out_dir"], graph)

    with tf.name_scope("initialization") as scope:
      init_op = tf.group(tf.global_variables_initializer(),
        tf.local_variables_initializer())

## Train ICA model (if needed)

In [ ]:
if not params["load_chk"]:
  if not os.path.exists(params["out_dir"]):
    os.makedirs(params["out_dir"])
  if not os.path.exists(params["chk_dir"]):
    os.makedirs(params["chk_dir"])

  with tf.Session(graph=graph) as sess:
    sess.run(init_op,
    feed_dict={x:np.zeros([params["batch_size"]]+params["input_shape"],
      dtype=np.float32)})
    sess.graph.finalize() # Graph is read-only after this statement
    
    #fig, sub_ax = plt.subplots(1, 1, figsize=(10, 10))  
    #init_data = sess.run(phi)
    #init_data = ip.normalize_data_with_max(init_data)
    #plot_data = pf.pad_data(init_data.reshape((params["num_neurons"],
    #  params["patch_edge_size"], params["patch_edge_size"])))
    #im_ax = sub_ax.imshow(plot_data, cmap="Greys_r", interpolation="nearest")
    #sub_ax.tick_params(axis="both", bottom="off", top="off", left="off", right="off")
    #sub_ax.get_xaxis().set_visible(False)
    #sub_ax.get_yaxis().set_visible(False)
    #plt.show()
    for b_step in range(params["num_training_steps"]):
      input_data = data["train"].next_batch(params["batch_size"])[0]
      feed_dict = {x:input_data}
      sess.run(update_weights, feed_dict)

      current_step = sess.run(global_step)
      if (current_step % params["update_interval"] == 0):
        summary = sess.run(merged_summaries, feed_dict)
        train_writer.add_summary(summary, current_step)
        full_saver.save(sess, save_path=params["chk_dir"]+"ica_chk", global_step=global_step)
        #weights = sess.run(phi, feed_dict)
        #weights = ip.normalize_data_with_max(weights)
        #plot_data = pf.pad_data(weights.reshape((params["num_neurons"],
        #  params["patch_edge_size"], params["patch_edge_size"])))
        #im_ax.set_data(plot_data)
        #sub_ax.set_title("Basis Functions at Time Step "+str(current_step), fontsize=24)  
        #fig.canvas.draw()
    full_saver.save(sess, save_path=params["chk_dir"]+"ica_chk",
      global_step=global_step)

## Compute activity covariance & pooling filters

### Compute activity covariance matrix

In [ ]:
if params["load_cov"]:
  u_cov = np.load(params["out_dir"]+"u_cov.npz")["data"]
  weights = np.load(params["out_dir"]+"weights.npz")["data"]
else:
  with tf.Session(graph=graph) as sess:
    sess.run(init_op,
      feed_dict={x:np.zeros([params["batch_size"]]+params["input_shape"],
      dtype=np.float32)})
    full_saver.restore(sess, tf.train.latest_checkpoint(params["chk_dir"]))
    weights = sess.run(phi).T
    u_cov = None
    num_cov_in_avg = 0
    tot_images = 0
    for tot_images in nb.log_progress(range(0, params["cov_num_images"], params["batch_size"]), every=1):
      input_data = data["train"].next_batch(params["batch_size"])[0]
      feed_dict = {x:input_data}
      if u_cov is None:
        u_cov = sess.run(u_covariance, feed_dict)
      else:
        u_cov += sess.run(u_covariance, feed_dict)
      num_cov_in_avg += 1
    u_cov /= num_cov_in_avg
  np.savez(params["out_dir"]+"u_cov.npz", data=u_cov)
  np.savez(params["out_dir"]+"weights.npz", data=weights)
#import scipy.io as sio
#sio.savemat(params["out_dir"]+"u_cov.mat", {"u_cov":u_cov})
#sio.savemat(params["out_dir"]+"phi_weights.mat", {"weights":weights})
u_evals, u_evecs = np.linalg.eig(u_cov)
u_sort_indices = np.argsort(u_evals)[::-1]

### Compute pooling filters and basis function summaries

In [ ]:
pooling_filters = compute_pooling_filters(u_cov, params["num_pooling_dims"])

In [ ]:
bf_stats = ip.get_dictionary_stats(weights, padding=20, num_gauss_fits=20, gauss_thresh=0.3)

## Construct analysis plots

### Covariance visualizations

In [ ]:
fig = pf.plot_cov_matrix(u_cov, str(params["cov_num_images"]))

In [ ]:
fig = pf.plot_eigenvalues(u_evals[u_sort_indices], ylim=[0, 100])

In [ ]:
num_random_bases = 8
num_top_cov_bases = 10
bf_indices = np.random.choice(np.arange(u_cov.shape[0]), num_random_bases)
fig = pf.plot_top_bases(u_cov, weights, bf_indices, num_top_cov_bases)

### Basis function analysis

In [ ]:
fig = pf.plot_hilbert_analysis(weights, params["ft_padding"])

In [ ]:
fig = pf.plot_loc_freq_summary(bf_stats)

In [ ]:
fig = pf.plot_bf_stats(bf_stats, num_bf=10)

In [ ]:
fig = pf.plot_gaussian_contours(bf_stats, num_plots=49)

### Pooling and eigen summary plots

In [ ]:
fig = pf.plot_ellipse_summaries(bf_stats, num_bf=params["num_neurons"], lines=True)

In [ ]:
params["num_pooling_filters"] = 64
params["num_connected_weights"] = params["num_neurons"]#250#75
fig = pf.plot_pooling_summaries(bf_stats, u_evecs[:, u_sort_indices[:params["num_pooling_filters"]]],
  params["num_pooling_filters"], params["num_connected_weights"], lines=True)

In [ ]:
params["num_pooling_filters"] = 36
params["num_connected_weights"] = params["num_neurons"]#250#75
fig = pf.plot_pooling_summaries(bf_stats, pooling_filters, params["num_pooling_filters"],
  params["num_connected_weights"], lines=True)

In [ ]:
num_pooling_filters = 16
fig_size = (10, 10)
spot_size = 100
fig = pf.plot_pooling_centers(bf_stats, pooling_filters, num_pooling_filters, fig_size, spot_size)